# Generate Topic Models
Generates the topic models of forum posts with LDA (Latent Dirichlet Allocation)

## Data Sources
- corpus (created with 3.0-Topic_Models-Lemmatize_Text.ipynb)
- dictionary (created with 3.0-Topic_Models-Lemmatize_Text.ipynb)
- lemmatized_text (created with 3.0-Topic_Models-Lemmatize_Text.ipynb)

## Changes
- 2020-09-16: Created
- 2020-09-17: Found topic model with highest coherence and generated dominant topics
- 2020-12-19: Added new data

## TODO
- Tutorial
 - https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21
 - https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python


## Imports

In [1]:
from gensim import corpora, models
import pickle
from pathlib import Path
from io import FileIO
import pyLDAvis.gensim
from gensim.models import CoherenceModel
import pandas as pd
from youbemom import create_connection
import csv

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/home/mwh/miniconda3/envs/forum/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Functions

In [3]:
def format_topics_sentences(ldamodel, corpus):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    return(sent_topics_df)

In [4]:
def write_coherence(fn, n_workers, n_topics, coherence, perplexity):
    with open(fn, 'a') as f:
        writer = csv.writer(f) 
        writer.writerow([n_workers, n_topics, coherence, perplexity])

In [5]:
def write_sample(fn, n, coherence, perplexity):
    with open(fn, 'a') as f:
        writer = csv.writer(f) 
        writer.writerow([n, coherence, perplexity])

## File Locations

In [6]:
p = Path.cwd()
path_parent = p.parents[0]

In [7]:
path_lemma_pkl = str(path_parent / "clean_data" / "lemmatized_text_{}.pkl")
path_corpus_pkl = str(path_parent / "clean_data" / "corpus_{}.pkl")
path_dictionary_gensim = str(path_parent / "clean_data" / "dictionary_{}.gensim")
path_model = path_parent / "clean_data"
path_sample = path_parent / "clean_data" / "sample_lda"
# path_db = path_parent / "database" / "youbemomTables.db"
# path_db = str(path_db)
path_coherence = str(path_parent / "clean_data" / "coherence_{}.csv")

## Special Needs

### Load Data

In [8]:
lemmatized_text = pickle.load(open(path_lemma_pkl.format('special-needs_parents'), 'rb'))

In [9]:
corpus = pickle.load(open(path_corpus_pkl.format('special-needs_parents'), 'rb'))

In [10]:
dictionary = corpora.Dictionary.load(path_dictionary_gensim.format('special-needs_parents'))

### Perform LDA
Identify the model with the highest coherence

In [ ]:
NUM_WORDS = 7
n_workers = [7,6,5,4,3,2,1,0]
n_topics = [2,3,4,5,6,7,8,9]
for t in n_topics:
    for w in n_workers:
        n = "model_LDA_{}_{}".format(str(t), "special-needs")
        fn = n + ".gensim"
        if w == 0:
            %time ldamodel = models.ldamodel.LdaModel(corpus, num_topics = t, id2word=dictionary, passes=15)
        else:
            %time ldamodel = models.LdaMulticore(corpus, num_topics = t, id2word=dictionary, passes=15, workers=w)
        topics = ldamodel.print_topics(num_words=NUM_WORDS)
        perplexity = ldamodel.log_perplexity(corpus)
        coherence_model = CoherenceModel(model=ldamodel, texts=lemmatized_text, dictionary=dictionary, coherence='c_v')
        coherence = coherence_model.get_coherence()
        print("LDA with {} topics and {} workers".format(t, w))
        for topic in topics:
            print(topic)
        print("Coherence: {}, Perplexity {}".format(coherence, perplexity))
        write_coherence(path_coherence.format("special-needs"), w, t, coherence, perplexity)

In [ ]:
NUM_WORDS = 7
n_workers = [9,8]
n_topics = [2,3,4,5,6,7,8,9]
for t in n_topics:
    for w in n_workers:
        if w == 0:
            %time ldamodel = models.ldamodel.LdaModel(corpus, num_topics = t, id2word=dictionary, passes=15)
        else:
            %time ldamodel = models.LdaMulticore(corpus, num_topics = t, id2word=dictionary, passes=15, workers=w)
        topics = ldamodel.print_topics(num_words=NUM_WORDS)
        perplexity = ldamodel.log_perplexity(corpus)
        coherence_model = CoherenceModel(model=ldamodel, texts=lemmatized_text, dictionary=dictionary, coherence='c_v')
        coherence = coherence_model.get_coherence()
        print("LDA with {} topics and {} workers".format(t, w))
        for topic in topics:
            print(topic)
        print("Coherence: {}, Perplexity {}".format(coherence, perplexity))
        write_coherence(path_coherence.format("special-needs"), w, t, coherence, perplexity)

In [11]:
NUM_WORDS = 7
NUM_WORKERS = 9
for t in range(2,31):
    for s in range(20):
        %time ldamodel = models.LdaMulticore(corpus, num_topics=t, id2word=dictionary, passes=15, workers=NUM_WORKERS)
        topics = ldamodel.print_topics(num_words=NUM_WORDS)
        perplexity = ldamodel.log_perplexity(corpus)
        coherence_model = CoherenceModel(model=ldamodel, texts=lemmatized_text, dictionary=dictionary, coherence='c_v')
        coherence = coherence_model.get_coherence()
        print("LDA with {} topics, sample {}".format(t, s))
        for topic in topics:
            print(topic)
        print("Coherence: {}, Perplexity {}".format(coherence, perplexity))
        write_coherence(path_coherence.format("special-needs_parents"), s, t, coherence, perplexity)

CPU times: user 30.9 s, sys: 4.89 s, total: 35.8 s
Wall time: 31 s
LDA with 2 topics, sample 0
(0, '0.018*"anyone" + 0.016*"school" + 0.014*"dc" + 0.009*"get" + 0.008*"child" + 0.007*"need" + 0.007*"know"')
(1, '0.017*"school" + 0.015*"ds" + 0.014*"kid" + 0.011*"get" + 0.009*"like" + 0.008*"adhd" + 0.007*"say"')
Coherence: 0.46128201865941254, Perplexity -7.591709178283438
CPU times: user 31.1 s, sys: 4.17 s, total: 35.3 s
Wall time: 31.1 s
LDA with 2 topics, sample 1
(0, '0.016*"anyone" + 0.012*"school" + 0.012*"dc" + 0.011*"ds" + 0.010*"child" + 0.009*"adhd" + 0.008*"kid"')
(1, '0.020*"school" + 0.013*"get" + 0.012*"kid" + 0.011*"ds" + 0.008*"need" + 0.008*"like" + 0.007*"dc"')
Coherence: 0.4697794095969119, Perplexity -7.61331034471811
CPU times: user 30.5 s, sys: 4.13 s, total: 34.6 s
Wall time: 30.2 s
LDA with 2 topics, sample 2
(0, '0.030*"school" + 0.011*"get" + 0.011*"kid" + 0.010*"dc" + 0.009*"sn" + 0.009*"need" + 0.008*"ds"')
(1, '0.015*"ds" + 0.009*"get" + 0.009*"like" + 0.0

CPU times: user 30.2 s, sys: 4.04 s, total: 34.3 s
Wall time: 30 s
LDA with 3 topics, sample 2
(0, '0.016*"ds" + 0.014*"kid" + 0.014*"school" + 0.012*"get" + 0.010*"like" + 0.008*"say" + 0.008*"dc"')
(1, '0.034*"school" + 0.012*"dc" + 0.012*"anyone" + 0.011*"get" + 0.009*"need" + 0.008*"ds" + 0.007*"iep"')
(2, '0.008*"anyone" + 0.008*"ds" + 0.008*"like" + 0.008*"kid" + 0.007*"dc" + 0.007*"get" + 0.007*"dd"')
Coherence: 0.47546787149996367, Perplexity -7.594983520480472
CPU times: user 29.9 s, sys: 4.6 s, total: 34.5 s
Wall time: 29.8 s
LDA with 3 topics, sample 3
(0, '0.016*"anyone" + 0.014*"adhd" + 0.013*"dc" + 0.012*"med" + 0.011*"ds" + 0.009*"help" + 0.008*"child"')
(1, '0.015*"kid" + 0.014*"ds" + 0.011*"get" + 0.009*"like" + 0.008*"school" + 0.007*"want" + 0.007*"say"')
(2, '0.035*"school" + 0.012*"get" + 0.011*"dc" + 0.009*"need" + 0.009*"sn" + 0.009*"anyone" + 0.008*"ds"')
Coherence: 0.503474808035149, Perplexity -7.571744024668594
CPU times: user 28.7 s, sys: 3.79 s, total: 32.5

LDA with 3 topics, sample 19
(0, '0.009*"ds" + 0.009*"get" + 0.008*"anyone" + 0.007*"kid" + 0.006*"reading" + 0.006*"dd" + 0.006*"know"')
(1, '0.015*"ds" + 0.013*"kid" + 0.011*"get" + 0.011*"like" + 0.010*"school" + 0.009*"dc" + 0.008*"adhd"')
(2, '0.033*"school" + 0.012*"anyone" + 0.011*"dc" + 0.011*"need" + 0.010*"get" + 0.009*"sn" + 0.008*"kid"')
Coherence: 0.4956123363660149, Perplexity -7.5797926291692965
CPU times: user 29.2 s, sys: 4.09 s, total: 33.3 s
Wall time: 29.2 s
LDA with 4 topics, sample 0
(0, '0.027*"anyone" + 0.024*"school" + 0.015*"kid" + 0.014*"dc" + 0.013*"child" + 0.009*"nyc" + 0.009*"need"')
(1, '0.012*"get" + 0.009*"school" + 0.008*"week" + 0.008*"want" + 0.008*"go" + 0.007*"would" + 0.007*"one"')
(2, '0.032*"school" + 0.013*"get" + 0.013*"sn" + 0.012*"ds" + 0.010*"need" + 0.009*"dc" + 0.008*"kid"')
(3, '0.016*"ds" + 0.011*"kid" + 0.011*"adhd" + 0.010*"dc" + 0.010*"get" + 0.009*"like" + 0.008*"med"')
Coherence: 0.527916082363006, Perplexity -7.607497154141269
CP

CPU times: user 28.3 s, sys: 3.96 s, total: 32.3 s
Wall time: 28.4 s
LDA with 4 topics, sample 14
(0, '0.013*"ds" + 0.012*"get" + 0.010*"like" + 0.008*"say" + 0.008*"kid" + 0.008*"im" + 0.007*"dd"')
(1, '0.032*"school" + 0.015*"get" + 0.013*"dc" + 0.012*"ds" + 0.012*"need" + 0.008*"would" + 0.007*"sn"')
(2, '0.024*"kid" + 0.013*"anyone" + 0.012*"school" + 0.010*"sn" + 0.010*"like" + 0.010*"dc" + 0.009*"child"')
(3, '0.019*"school" + 0.015*"anyone" + 0.015*"adhd" + 0.015*"dc" + 0.014*"med" + 0.013*"ds" + 0.011*"kid"')
Coherence: 0.48977516770506674, Perplexity -7.609367016155824
CPU times: user 26.2 s, sys: 3.63 s, total: 29.8 s
Wall time: 26.2 s
LDA with 4 topics, sample 15
(0, '0.017*"ds" + 0.011*"kid" + 0.011*"get" + 0.010*"like" + 0.008*"adhd" + 0.008*"im" + 0.007*"he"')
(1, '0.032*"school" + 0.023*"sn" + 0.020*"kid" + 0.014*"dc" + 0.012*"anyone" + 0.011*"child" + 0.008*"parent"')
(2, '0.032*"school" + 0.013*"get" + 0.011*"dc" + 0.011*"need" + 0.009*"anyone" + 0.009*"ds" + 0.009*"ie

CPU times: user 28.9 s, sys: 4.38 s, total: 33.3 s
Wall time: 29 s
LDA with 5 topics, sample 7
(0, '0.014*"get" + 0.014*"ds" + 0.012*"year" + 0.009*"school" + 0.008*"like" + 0.008*"im" + 0.007*"week"')
(1, '0.025*"anyone" + 0.008*"speech" + 0.008*"therapist" + 0.008*"child" + 0.008*"recommend" + 0.007*"services" + 0.007*"issue"')
(2, '0.048*"school" + 0.016*"sn" + 0.016*"kid" + 0.013*"dc" + 0.012*"anyone" + 0.011*"need" + 0.011*"child"')
(3, '0.016*"ds" + 0.014*"kid" + 0.011*"get" + 0.011*"like" + 0.008*"say" + 0.008*"school" + 0.008*"want"')
(4, '0.017*"dc" + 0.014*"adhd" + 0.013*"school" + 0.012*"med" + 0.011*"ds" + 0.010*"get" + 0.010*"anyone"')
Coherence: 0.5216541995902473, Perplexity -7.6329352961194195
CPU times: user 28.6 s, sys: 4.37 s, total: 33 s
Wall time: 28.8 s
LDA with 5 topics, sample 8
(0, '0.017*"adhd" + 0.016*"med" + 0.013*"ds" + 0.012*"dc" + 0.011*"anyone" + 0.010*"help" + 0.010*"take"')
(1, '0.015*"ds" + 0.013*"kid" + 0.012*"like" + 0.010*"get" + 0.009*"he" + 0.009

...

CPU times: user 27.7 s, sys: 4.78 s, total: 32.5 s
Wall time: 28.8 s
LDA with 23 topics, sample 9
(22, '0.034*"kid" + 0.023*"school" + 0.017*"anyone" + 0.015*"adhd" + 0.014*"like" + 0.011*"know" + 0.011*"mom"')
(0, '0.069*"school" + 0.026*"sn" + 0.016*"dc" + 0.016*"grade" + 0.012*"get" + 0.012*"ds" + 0.011*"private"')
(7, '0.028*"specialist" + 0.020*"love" + 0.012*"ds" + 0.012*"918728849451" + 0.009*"vashikaran" + 0.008*"ji" + 0.008*"marriage"')
(14, '0.059*"anyone" + 0.030*"nyc" + 0.026*"recommend" + 0.023*"tia" + 0.023*"camp" + 0.022*"therapist" + 0.021*"recommendation"')
(9, '0.018*"go" + 0.016*"dc" + 0.014*"ds" + 0.014*"anyone" + 0.012*"take" + 0.010*"try" + 0.008*"med"')
(13, '0.036*"med" + 0.031*"adhd" + 0.019*"ds" + 0.013*"dc" + 0.013*"help" + 0.011*"anxiety" + 0.010*"think"')
(18, '0.014*"ds" + 0.011*"get" + 0.009*"np" + 0.008*"us" + 0.008*"test" + 0.006*"one" + 0.006*"seem"')
(4, '0.027*"kid" + 0.017*"play" + 0.015*"like" + 0.014*"want" + 0.011*"one" + 0.011*"ds" + 0.011*"pare

CPU times: user 26.3 s, sys: 3.98 s, total: 30.3 s
Wall time: 27.4 s
LDA with 23 topics, sample 13
(20, '0.030*"med" + 0.024*"adhd" + 0.022*"dc" + 0.020*"ds" + 0.016*"try" + 0.015*"anxiety" + 0.014*"take"')
(14, '0.024*"love" + 0.022*"specialist" + 0.015*"918728849451" + 0.013*"anyone" + 0.009*"vashikaran" + 0.009*"ji" + 0.008*"marriage"')
(15, '0.058*"camp" + 0.031*"summer" + 0.020*"special" + 0.017*"week" + 0.015*"kid" + 0.014*"program" + 0.013*"need"')
(3, '0.017*"adhd" + 0.015*"school" + 0.014*"like" + 0.014*"help" + 0.014*"issue" + 0.013*"med" + 0.011*"he"')
(8, '0.063*"school" + 0.024*"dc" + 0.017*"sn" + 0.015*"nyc" + 0.013*"kid" + 0.013*"public" + 0.013*"anyone"')
(10, '0.026*"child" + 0.016*"anyone" + 0.015*"kid" + 0.013*"autism" + 0.012*"spectrum" + 0.010*"ds" + 0.010*"asd"')
(2, '0.022*"like" + 0.021*"year" + 0.017*"old" + 0.015*"kid" + 0.013*"adhd" + 0.012*"get" + 0.011*"feel"')
(7, '0.056*"anyone" + 0.023*"recommend" + 0.023*"nyc" + 0.022*"therapist" + 0.019*"tia" + 0.018*"

CPU times: user 26.4 s, sys: 4.36 s, total: 30.8 s
Wall time: 27.4 s
LDA with 23 topics, sample 17
(22, '0.019*"need" + 0.014*"anyone" + 0.013*"teacher" + 0.013*"get" + 0.011*"dd" + 0.010*"adhd" + 0.009*"help"')
(4, '0.028*"issue" + 0.018*"school" + 0.018*"ds" + 0.017*"dc" + 0.011*"anyone" + 0.010*"processing" + 0.009*"language"')
(18, '0.025*"camp" + 0.024*"kid" + 0.014*"ds" + 0.013*"get" + 0.012*"say" + 0.008*"tell" + 0.008*"day"')
(15, '0.019*"kid" + 0.010*"cbt" + 0.010*"anxiety" + 0.009*"anyone" + 0.009*"recommend" + 0.009*"sensory" + 0.008*"get"')
(14, '0.080*"school" + 0.022*"sn" + 0.014*"kid" + 0.014*"dc" + 0.013*"private" + 0.012*"get" + 0.011*"need"')
(11, '0.022*"love" + 0.018*"specialist" + 0.010*"problem" + 0.009*"918728849451" + 0.008*"life" + 0.007*"solution" + 0.007*"vashikaran"')
(2, '0.017*"child" + 0.014*"think" + 0.014*"know" + 0.011*"like" + 0.010*"adhd" + 0.010*"dont" + 0.009*"would"')
(19, '0.019*"child" + 0.015*"parent" + 0.010*"kid" + 0.009*"mom" + 0.009*"go" + 

CPU times: user 27 s, sys: 4.53 s, total: 31.6 s
Wall time: 28.1 s
LDA with 24 topics, sample 1
(15, '0.033*"med" + 0.027*"adhd" + 0.024*"anxiety" + 0.019*"dc" + 0.016*"ds" + 0.015*"anyone" + 0.011*"try"')
(16, '0.014*"teacher" + 0.014*"conference" + 0.012*"ds" + 0.008*"executive" + 0.007*"adderall" + 0.006*"function" + 0.005*"need"')
(19, '0.013*"get" + 0.012*"help" + 0.011*"time" + 0.011*"dc" + 0.010*"issue" + 0.009*"dd" + 0.008*"one"')
(8, '0.017*"time" + 0.016*"week" + 0.015*"get" + 0.012*"ds" + 0.012*"he" + 0.011*"take" + 0.009*"day"')
(9, '0.032*"kid" + 0.017*"school" + 0.017*"like" + 0.017*"dc" + 0.012*"feel" + 0.010*"sn" + 0.007*"group"')
(23, '0.012*"issue" + 0.010*"one" + 0.009*"like" + 0.008*"friend" + 0.008*"kid" + 0.008*"really" + 0.008*"child"')
(2, '0.015*"ds" + 0.015*"he" + 0.012*"weight" + 0.010*"like" + 0.009*"year" + 0.008*"say" + 0.008*"anyone"')
(17, '0.042*"school" + 0.027*"dc" + 0.016*"class" + 0.015*"sn" + 0.013*"get" + 0.011*"would" + 0.010*"need"')
(7, '0.041*

Process ForkPoolWorker-12413:
Process ForkPoolWorker-12410:
Process ForkPoolWorker-12411:
Process ForkPoolWorker-12408:
Process ForkPoolWorker-12412:
Process ForkPoolWorker-12406:
Process ForkPoolWorker-12409:
Process ForkPoolWorker-12405:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


Compare coherence and topics, run 100 times

In [ ]:
NUM_WORKERS = 9
NUM_TOPICS = 7
lda_models = {}
for i in range(100):
    fn = "model_{}.gensim".format(str(i))
    ldamodel = models.LdaMulticore(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15, workers=NUM_WORKERS)
    path_sample_i = path_sample / fn
    ldamodel.save(str(path_sample_i))
    perplexity = ldamodel.log_perplexity(corpus)
    coherence_model = CoherenceModel(model=ldamodel, texts=lemmatized_text, dictionary=dictionary, coherence='c_v')
    coherence = coherence_model.get_coherence()
    write_sample(path_coherence.format("sample"), i, coherence, perplexity)
    print("Coherence: {}, Perplexity {}".format(coherence, perplexity))

In [ ]:
n_topics = [6, 8]
for i in n_topics:
    n = "model_LDA_" + str(i)
    fn = n + ".gensim"
    ldamodel = models.ldamodel.LdaModel(corpus, num_topics = i, id2word=dictionary, passes=15)
    path_model_i = path_model / fn
    ldamodel.save(str(path_model_i))
    topics = ldamodel.print_topics(num_words=NUM_WORDS)
    perplexity = ldamodel.log_perplexity(corpus)
    coherence_model = CoherenceModel(model=ldamodel, texts=lemmatized_text, dictionary=dictionary, coherence='c_v')
    coherence = coherence_model.get_coherence()
    lda_models[n] = {'model' : ldamodel, 'coherence' : coherence, 'perplexity': perplexity}
    print("LDA with {} topics".format(i))
    for topic in topics:
        print(topic)
    print("Coherence: {}, Perplexity {}".format(coherence, perplexity))
    print("\n")

Visualize the topics. See: https://www.objectorientedsubject.net/2018/08/experiments-on-topic-modeling-pyldavis/

In [ ]:
pyLDAvis.display(pyLDAvis.gensim.prepare(lda_models["model_LDA_7"]["model"], corpus, dictionary, sort_topics=False))

## What is the Dominant Topic in each Post?

In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_models["model_LDA_7"]["model"], corpus=corpus)
df_topic_sents_keywords.info()
df_topic_sents_keywords.head()

## Save Model Topics and Keywords in New Database

In [ ]:
conn = sqlite3.connect(path_db)
df_topic_sents_keywords.to_sql('topicmodel', conn, if_exists='replace', index=False)